# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 9:19AM,263457,10,MSP222052-Hold_Rx(MN),"Methapharm, Inc.",Released
1,May 23 2023 9:17AM,263456,20,9505816,"Exact-Rx, Inc.",Released
2,May 23 2023 9:17AM,263456,20,9505817,"Exact-Rx, Inc.",Released
3,May 23 2023 9:17AM,263456,20,9505818,"Exact-Rx, Inc.",Released
4,May 23 2023 9:17AM,263456,20,9505819,"Exact-Rx, Inc.",Released
5,May 23 2023 9:17AM,263456,20,9505820,"Exact-Rx, Inc.",Released
6,May 23 2023 9:17AM,263456,20,9505821-Hold_Rx(AL),"Exact-Rx, Inc.",Released
7,May 23 2023 9:17AM,263456,20,9505822,"Exact-Rx, Inc.",Released
8,May 23 2023 9:17AM,263456,20,9505823,"Exact-Rx, Inc.",Released
9,May 23 2023 9:17AM,263456,20,9505824,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,263453,Released,2
20,263454,Released,1
21,263455,Released,14
22,263456,Released,19
23,263457,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
263453,NaN,2.0
263454,NaN,1.0
263455,NaN,14.0
263456,NaN,19.0
263457,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
263367,67.0,0.0
263369,1.0,0.0
263420,3.0,2.0
263434,13.0,7.0
263439,10.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
263367,67,0
263369,1,0
263420,3,2
263434,13,7
263439,10,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,263367,67,0
1,263369,1,0
2,263420,3,2
3,263434,13,7
4,263439,10,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,263367,67,
1,263369,1,
2,263420,3,2
3,263434,13,7
4,263439,10,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 9:19AM,263457,10,"Methapharm, Inc."
1,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc."
20,May 23 2023 9:02AM,263455,10,Hush Hush
34,May 23 2023 8:59AM,263454,10,Hush Hush
35,May 23 2023 8:54AM,263453,10,"Digital Brands, LLC"
37,May 23 2023 8:46AM,263452,19,"AdvaGen Pharma, Ltd"
44,May 23 2023 8:28AM,263450,16,Sartorius Lab Products and Service
51,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc."
52,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC
75,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",,1
1,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",,19
2,May 23 2023 9:02AM,263455,10,Hush Hush,,14
3,May 23 2023 8:59AM,263454,10,Hush Hush,,1
4,May 23 2023 8:54AM,263453,10,"Digital Brands, LLC",,2
5,May 23 2023 8:46AM,263452,19,"AdvaGen Pharma, Ltd",,7
6,May 23 2023 8:28AM,263450,16,Sartorius Lab Products and Service,,7
7,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",,1
8,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,6,17
9,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",1,
1,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",19,
2,May 23 2023 9:02AM,263455,10,Hush Hush,14,
3,May 23 2023 8:59AM,263454,10,Hush Hush,1,
4,May 23 2023 8:54AM,263453,10,"Digital Brands, LLC",2,
5,May 23 2023 8:46AM,263452,19,"AdvaGen Pharma, Ltd",7,
6,May 23 2023 8:28AM,263450,16,Sartorius Lab Products and Service,7,
7,May 23 2023 8:24AM,263449,19,"GCH Granules USA, Inc.",1,
8,May 23 2023 8:24AM,263448,20,Reliable 1 Laboratories LLC,17,6
9,May 23 2023 8:11AM,263445,22,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",1,
1,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",19,
2,May 23 2023 9:02AM,263455,10,Hush Hush,14,
3,May 23 2023 8:59AM,263454,10,Hush Hush,1,
4,May 23 2023 8:54AM,263453,10,"Digital Brands, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",1.0,NaN
1,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",19.0,NaN
2,May 23 2023 9:02AM,263455,10,Hush Hush,14.0,NaN
3,May 23 2023 8:59AM,263454,10,Hush Hush,1.0,NaN
4,May 23 2023 8:54AM,263453,10,"Digital Brands, LLC",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",1.0,0.0
1,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",19.0,0.0
2,May 23 2023 9:02AM,263455,10,Hush Hush,14.0,0.0
3,May 23 2023 8:59AM,263454,10,Hush Hush,1.0,0.0
4,May 23 2023 8:54AM,263453,10,"Digital Brands, LLC",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1844047,20.0,71.0
16,263450,7.0,0.0
19,1317227,75.0,25.0
20,790338,43.0,19.0
22,526814,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1844047,20.0,71.0
1,16,263450,7.0,0.0
2,19,1317227,75.0,25.0
3,20,790338,43.0,19.0
4,22,526814,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,20.0,71.0
1,16,7.0,0.0
2,19,75.0,25.0
3,20,43.0,19.0
4,22,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,16,Released,7.0
2,19,Released,75.0
3,20,Released,43.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Executing,71.0,0.0,25.0,19.0,2.0
Released,20.0,7.0,75.0,43.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Executing,71.0,0.0,25.0,19.0,2.0
1,Released,20.0,7.0,75.0,43.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Executing,71.0,0.0,25.0,19.0,2.0
1,Released,20.0,7.0,75.0,43.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()